# Module 20: ADAS Foundations (No-Code Lab)

This module is independent from the coding track.
You are learning how ADAS works on the Lectec platform by operating a complete system, not by writing code.

## Section 1: What You Are Building

In this track, your ADAS stack is:
- **AI camera** for object detection
- **Confidence filtering** to reject weak detections
- **Buzzer** for audible warnings
- **VESC telemetry** for speed-aware safety logic

You will tune variables and evaluate safety behavior.

## Section 2: Startup (Prebuilt)

Run the next cell once. If the camera is busy, the API will auto-recover stale sessions.

In [2]:
import sys, os, time, cv2
from IPython.display import display, Image, clear_output
sys.path.append(os.path.abspath('../'))
from student_api import AIStudentAPI, VESCStudentAPI

ai_api = AIStudentAPI()
if not ai_api.start_camera():
    raise RuntimeError('Camera failed to start. Re-run this cell.')
if not ai_api.start_buzzer():
    raise RuntimeError('Failed to initialize buzzer on GPIO17')

vesc_api = VESCStudentAPI()
vesc = None
if vesc_api.start():
    deadline = time.time() + 10.0
    controllers = []
    while time.time() < deadline and not controllers:
        controllers = vesc_api.get_connected_controllers()
        if not controllers:
            time.sleep(0.5)
    if controllers:
        vesc = vesc_api.get_controller(controllers[0])

print('✅ ADAS lab ready')
print('VESC connected:', bool(vesc))

[4:43:46.762715688] [8761]  INFO Camera camera_manager.cpp:326 libcamera v0.5.0+59-d83ff0a4
[4:43:47.582121111] [8804]  WARN RPiSdn sdn.cpp:40 Using legacy SDN tuning - please consider moving SDN inside rpi.denoise
[4:43:47.589395631] [8804]  INFO RPI vc4.cpp:447 Registered camera /base/soc/i2c0mux/i2c@1/imx500@1a to Unicam device /dev/media3 and ISP device /dev/media0
[4:43:47.589532610] [8804]  INFO RPI pipeline_base.cpp:1121 Using configuration file '/usr/share/libcamera/pipeline/rpi/vc4/rpi_apps.yaml'

------------------------------------------------------------------------------------------------------------------
NOTE: Loading network firmware onto the IMX500 can take several minutes, please do not close down the application.
------------------------------------------------------------------------------------------------------------------

Network Firmware Upload: 0.00bytes [00:00, ?bytes/s][4:43:48.672933879] [8761]  INFO Camera camera.cpp:1205 configuring streams: (0) 640x480-X

Starting VESC CAN System...
Connected to CAN bus: can0
Main processing loop started
VESC CAN System started successfully
✅ ADAS lab ready
VESC connected: False


## Section 3: Detectable Object Library

The Lectec ADAS model can detect many COCO object classes. Run the next cell to print all supported labels.

In [3]:
labels = sorted([label for label in AIStudentAPI.COCO_LABELS if label])
print(f'Total detectable labels: {len(labels)}')
print(', '.join(labels))

Total detectable labels: 80
airplane, apple, backpack, banana, baseball bat, baseball glove, bear, bed, bench, bicycle, bird, boat, book, bottle, bowl, broccoli, bus, cake, car, carrot, cat, cell phone, chair, clock, couch, cow, cup, dining table, dog, donut, elephant, fire hydrant, fork, frisbee, giraffe, hair drier, handbag, horse, hot dog, keyboard, kite, knife, laptop, microwave, motorcycle, mouse, orange, oven, parking meter, person, pizza, potted plant, refrigerator, remote, sandwich, scissors, sheep, sink, skateboard, skis, snowboard, spoon, sports ball, stop sign, suitcase, surfboard, teddy bear, tennis racket, tie, toaster, toilet, toothbrush, traffic light, train, truck, tv, umbrella, vase, wine glass, zebra


## Section 4: Adjustable Parameters

Change values only in this cell. You do not need to edit any other code.

In [ ]:
# Object selection
TARGET_LABEL = 'person'

# Detection quality threshold
MIN_CONFIDENCE = 0.70

# Speed gating: alerts only trigger when RPM is above this value
MIN_RPM_FOR_ALERT = 300

# Alert behavior
BEEP_COOLDOWN_S = 4.0
RUN_DELAY_S = 0.12

## Section 5: Live Detection Monitor

In [ ]:
last_beep_time = 0.0

try:
    print('Running monitor... press Interrupt (■) to stop.')
    while True:
        frame, detections = ai_api.get_frame_and_detections()
        rpm = vesc.get_rpm() if vesc else 0.0
        abs_rpm = abs(rpm)

        target_detected = False
        top_conf = 0.0
        for det in detections:
            x, y, w, h = det['box']
            label = det['label']
            conf = float(det['confidence'])
            top_conf = max(top_conf, conf)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (4, 23, 115), 2)
            cv2.putText(frame, f"{label}: {conf:.2f}", (x, max(15, y - 5)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (4, 23, 115), 2)
            if label == TARGET_LABEL and conf >= MIN_CONFIDENCE:
                target_detected = True

        speed_gate_pass = abs_rpm >= MIN_RPM_FOR_ALERT

        now = time.time()
        if target_detected and speed_gate_pass and now - last_beep_time >= BEEP_COOLDOWN_S:
            ai_api.buzzer_beep(0.08, 0.08, 2)
            last_beep_time = now

        status = (
            f"Target={TARGET_LABEL} | Detected={target_detected} | "
            f"ConfMax={top_conf:.2f} | RPM={rpm:.0f} | RPMGate={speed_gate_pass}"
        )
        cv2.putText(frame, status, (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.50, (255, 255, 255), 2)

        _, img_encoded = cv2.imencode('.jpeg', frame)
        clear_output(wait=True)
        display(Image(data=img_encoded.tobytes()))
        print(status)
        time.sleep(RUN_DELAY_S)

except KeyboardInterrupt:
    print('
Stopped.')

## Section 6: ADAS Knowledge Check (No Coding)

1. Why do we use a confidence threshold before triggering alerts?
2. Why do we apply an RPM gate before warning the rider?
3. Pick one non-person object from the library and explain when it could matter for safety.

In [ ]:
if 'ai_api' in locals():
    ai_api.shutdown_hardware(stop_camera=True, stop_buzzer=True)
    print('Camera + buzzer released.')
if 'vesc_api' in locals() and vesc_api.is_running():
    vesc_api.stop()
    print('VESC API closed.')